In [1]:
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
DEV = False

if DEV:
    model_name = "avsolatorio/GIST-small-Embedding-v0"
else:
    model_name = "avsolatorio/GIST-Embedding-v0"

In [3]:
def import_labelled_data(path="data/level-0.5/data.json"):
    data = pd.read_json(path, encoding="latin-1")
    return data


data = import_labelled_data(path="../../../data/level-0.5/data.json", )



# train test split
from sklearn.model_selection import train_test_split

# 0.49, 0.21, 0.3 split
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.3, random_state=42)

display(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20901 entries, 0 to 20900
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   url           20901 non-null  object
 1   text          20901 non-null  object
 2   relevance     20901 non-null  object
 3   multiclasses  20901 non-null  object
dtypes: object(4)
memory usage: 653.3+ KB


None

,url,text,relevance,multiclasses
0,https://www.conservationevidence.com/synopsis/...,1 \n \n \n2 \n \n \nSubtidal Benthic Invertebr...,relevant,[Marine Invertebrates]
1,https://www.conservationevidence.com/synopsis/...,\n \n \n Control of freshwater \n invasi...,relevant,"[Fish, Rivers and Lakes, Invasive]"
2,https://www.conservationevidence.com/synopsis/...,1 \n \nGrassland Conservation \n2 \n \nGrassla...,relevant,[Grassland]
3,https://www.conservationevidence.com/synopsis/...,\n \n \nii \n \n \n \n \n \n \n \nPrimate Co...,relevant,[Mammals]
4,https://www.conservationevidence.com/synopsis/...,CONSERVATION EVIDENCE SERIES SYNOPSES\nTerrest...,relevant,[Mammals]


In [4]:
from chunking import chunk_dataset_and_explode


# roughly 4 characters per token
max_len = 512

train_data = chunk_dataset_and_explode(train_data, max_len=max_len, overlap=int(max_len * 0.2))
test_data = chunk_dataset_and_explode(test_data, max_len=max_len, overlap=int(max_len * 0.2))
val_data = chunk_dataset_and_explode(val_data, max_len=max_len, overlap=int(max_len * 0.2))

  0%|          | 0/10241 [00:00<?, ?it/s]

  0%|          | 0/6271 [00:00<?, ?it/s]

  0%|          | 0/4389 [00:00<?, ?it/s]

In [5]:
def stratified_sample(dataset,label_column: str = 'relevance',num_samples_per_label: int = 100):
    return (
        dataset
        .sample(frac=1,random_state=42)
        .groupby(label_column)[dataset.columns]
        .apply(lambda x: x.sample(min(num_samples_per_label,len(x)),random_state=42),include_groups=True).reset_index(drop=True)
    )

if DEV:
    train_data = stratified_sample(train_data,num_samples_per_label=250)
    val_data = val_data.sample(100,random_state=42)
    test_data = test_data.sample(200,random_state=42)
else:
    train_data = train_data.sample(frac=1,random_state=42)
    val_data = val_data.sample(500,random_state=42)
    test_data = test_data.sample(frac=1,random_state=42)


In [6]:
train_data

,chunk_id,url,text,relevance,multiclasses
659800,18518,https://phlconnect.ched.gov.ph/admin/uploads/a...,: • Atomic operations • Reserve/publish • Tran...,irrelevant,[]
536581,17275,https://repositorio.upn.edu.pe/bitstream/handl...,ra y entendible para los profesionales e invol...,irrelevant,[]
1575204,10748,https://repositorio.unfv.edu.pe/bitstream/hand...,isión del espacio en zonas para cada grupo de ...,irrelevant,[]
263202,12009,https://repositorio.upn.edu.pe/bitstream/handl...,ón Definición Dimensiones Indicadores Ítems co...,irrelevant,[]
19541,19307,https://www.yololafco.org/files/f96ec5765/LAFC...,"the law of the State of California, and Magell...",irrelevant,[]
...,...,...,...,...,...
259178,12660,https://www.wjec.paris/wp-content/uploads/2020...,ying to help communities develop civic capacit...,irrelevant,[]
1414414,16282,https://www.investinperu.pe/RepositorioAPS0/0/...,"2 , cI ul uI uuul L2 LR0 SBg2 ~ c cNP 7 ORLLR2...",irrelevant,[]
131932,10649,https://ymcamalta.org/wp-content/uploads/2024/...,ys the social network of the affected individu...,irrelevant,[]
671155,19384,https://documents1.worldbank.org/curated/en/16...,a capacity assessment study and select trainin...,irrelevant,[]


In [7]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data, split="train")
test_dataset = Dataset.from_pandas(test_data, split="test")
val_dataset = Dataset.from_pandas(val_data, split="val")

train_dataset

Dataset({
    features: ['chunk_id', 'url', 'text', 'relevance', 'multiclasses', '__index_level_0__'],
    num_rows: 1660297
})

In [8]:

#! had to modify FastFitTrainer to at /fastfit/train.py, line 879, to add trust_remote_code=True to the loading of 'accuracy' metrics
#! don't know why it's not default, since accuracy is the default in fastfit



#! IMPORTANT: another change in FastFitTrainer, also at line 879; comment out and replace the fixed version above
#! since load_metric is deprecated in favour of evaluate.load()
#! added functionality for sending in multiple metrics to evaluate at once
#! added macro averages for non-accuracy metrics too
#! essentially, copy the below code to replace line 879

: 

### Insert into line 879.

```python
        # metric = load_metric(self.data_args.metric_name, experiment_id=uuid.uuid4())
        from evaluate import combine, load
        if type(self.data_args.metric_name) == str: # single metric name
            metrics = [load(self.data_args.metric_name, experiment_id=uuid.uuid4())]
        elif type(self.data_args.metric_name) == list: # compute multiple metrics
            metrics = [load(metric,experiment_id=uuid.uuid4()) for metric in self.data_args.metric_name]

        # You can define your custom compute_metrics function. It takes an `EvalPrediction` object (a namedtuple with a
        # predictions and label_ids field) and has to return a dictionary string to float.
        def compute_metrics(p: EvalPrediction):
            predictions = (
                p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
            )
            predictions = (
                np.squeeze(predictions)
                if self.is_regression
                else np.argmax(predictions, axis=1)
            )
            references = p.label_ids

            results = {}

            for metric in metrics:
                if metric.name != 'accuracy':
                    results.update(metric.compute(predictions=predictions, references=references,average='macro'))
                else:
                    results.update(metric.compute(predictions=predictions, references=references))

            return results
    ```

In [9]:
from fastfit import FastFitTrainer

# same args as the huggingface TrainingArguments
if DEV:
    output_dir = f'models/relevance/dev/{model_name}'
else:
    output_dir = f'models/relevance/{model_name}'

trainer = FastFitTrainer(
    model_name_or_path=model_name,
    train_dataset=train_dataset,
    validation_dataset=val_dataset,
    test_dataset=test_dataset,
    output_dir=output_dir,
    overwrite_output_dir=True,
    label_column_name='relevance',
    text_column_name="text",
    num_train_epochs=10,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    max_text_length=512,
    num_repeats=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy='epoch',
    metric_name=['precision','recall','f1','accuracy'],
    load_best_model_at_end=True,
    metric_for_best_model='precision',
    fp16=True,
    fp16_opt_level="O2",
    fp16_full_eval=True,
)

07/29/2024 11:14:40 - WARNING - fastfit.train - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: True


/root/.local/share/virtualenvs/GreyLiteratureClassifier-eJH_GeT1/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Running tokenizer on dataset to infer max length for both query and document:   0%|          | 0/1660297 [00:0…

Running tokenizer on dataset to infer max length for both query and document:   0%|          | 0/500 [00:00<?,…

Running tokenizer on dataset to infer max length for both query and document:   0%|          | 0/1001094 [00:0…

Running tokenizer on dataset:   0%|          | 0/1660297 [00:00<?, ? examples/s]

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

print(torch.version.cuda)

torch.cuda.empty_cache()
print(trainer.model.device)

True
0
NVIDIA GeForce RTX 3090
12.1
cuda:0


In [ ]:

#! another fastfit library modification
#! in /fastfit/train.py, line 971, change ignore_keys_for_eval from type set to a list
#! since it gets concatenated to a list later on
#! note that since we've added lines above, this is now line 981
#! the line beginning ignore_keys_for_eval={"doc_input_ids","doc_attention_mask","labels"}


model = trainer.train()

[WARNING|modeling_utils.py:1198] 2024-07-29 09:18:11,707 >> Could not estimate the number of tokens of the input, floating-point operations will not be computed


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,4.875100,4.793930,0.676471,0.941489,0.729796,0.890000
2,4.362100,4.728496,0.730769,0.962766,0.796452,0.930000
3,4.257300,4.705177,0.730769,0.962766,0.796452,0.930000
4,4.213300,4.717960,0.730769,0.962766,0.796452,0.930000
5,4.186000,4.773394,0.730769,0.962766,0.796452,0.930000
6,4.167800,4.733587,0.772727,0.973404,0.839280,0.950000
7,4.170700,4.849503,0.730769,0.962766,0.796452,0.930000
8,4.153600,4.907688,0.730769,0.962766,0.796452,0.930000
9,4.155000,4.913159,0.730769,0.962766,0.796452,0.930000


KeyboardInterrupt: 

In [ ]:
results = trainer.evaluate()

***** eval metrics *****
  epoch                   =       10.0
  eval_accuracy           =       0.94
  eval_f1                 =     0.8168
  eval_loss               =     4.7342
  eval_precision          =       0.75
  eval_recall             =     0.9681
  eval_runtime            = 0:00:00.76
  eval_samples            =        100
  eval_samples_per_second =     131.48
  eval_steps_per_second   =       2.63


In [ ]:
results

{'eval_loss': 4.734198093414307,
 'eval_precision': 0.75,
 'eval_recall': 0.9680851063829787,
 'eval_f1': 0.8168498168498168,
 'eval_accuracy': 0.94,
 'eval_runtime': 0.7606,
 'eval_samples_per_second': 131.48,
 'eval_steps_per_second': 2.63,
 'epoch': 10.0,
 'eval_samples': 100}

In [ ]:
print(f'Accuracy: {results["eval_accuracy"]}')

Accuracy: 0.94


In [ ]:
results = trainer.test()

***** test metrics *****
  epoch                   =       10.0
  eval_accuracy           =      0.935
  eval_f1                 =     0.7583
  eval_loss               =      4.914
  eval_precision          =     0.6972
  eval_recall             =      0.913
  eval_runtime            = 0:00:01.27
  eval_samples_per_second =    156.795
  eval_steps_per_second   =      3.136
  test_samples            =        200


In [ ]:

if DEV:
    from os import makedirs, path
    if not path.exists(f'models/relevance/dev/{model_name}'):
        makedirs(f'models/relevance/dev/{model_name}')

    model.save_pretrained(f'models/relevance/dev/{model_name}')

else:
    model.save_pretrained(f'models/relevance/{model_name}')

In [ ]:
from FastFitClassifier import FastFitClassifier

classifier = FastFitClassifier(model_path=f"models/relevance/dev/{model_name}",tokenizer=model_name)